### AIR Project

## Imports and specific settings

In [271]:
import ast
import string

import torch
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from gensim.models import Word2Vec
import multiprocessing
import time
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

In [272]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True
print("running models with {}".format(device))

running models with cuda:0


## Loading of dataset and nan-removal

In [273]:
def pre_process():
    data = pd.read_csv('WELFake_Dataset.csv', index_col=0)
    print(data.shape)
    # display(data[:300])
    for i,x in data.iterrows():
        if len(str(x["text"])) <= 10:
            data.loc[i, "text"] = np.nan
        if len(str(x["title"])) <= 10:
            data.loc[i, "title"] = np.nan

    data.dropna(inplace=True)
    print(data.shape)
    data.reset_index(drop=True, inplace=True)
    data.to_csv("data/data.csv")
    display(data[:300])

## Tokenization

In [274]:
def tokenize():
    stop = stopwords.words('english')
    stemmer = SnowballStemmer('english')
    punc = [u'\u201c',u'\u201d',u'\u2018',u'\u2019',u'\u2024',u'\u2025',u'\u2026',u'\u2027']
    # print(punc)
    data = pd.read_csv('data/data.csv', index_col=0)
    titles = list()
    texts = list()
    for i, row in data.iterrows():
        title = str(row["title"])
        text = str(row["text"])
        t1 = ""
        for c in title:
            if not (c in string.punctuation or c in punc):
                t1 += c
            else:
                t1 += " "
        t2 = ""
        for c in text:
            if not (c in string.punctuation or c in punc):
                t2 += c
            else:
                t2 += " "
        title_tokens = nltk.tokenize.word_tokenize(t1)
        text_tokens = nltk.tokenize.word_tokenize(t2)
        # title_filtered = [w.lower() for w in title_tokens if not w.lower() in string.punctuation]
        # title_filtered = [w.lower() for w in title_filtered if not w.lower() in punc]
        title_filtered = [w.lower() for w in title_tokens if not w.lower() in stop]
        title_stemmed = [stemmer.stem(w) for w in title_filtered]
        # text_filtered = [w.lower() for w in text_tokens if not w.lower() in string.punctuation]
        # text_filtered = [w.lower() for w in text_filtered if not w.lower() in punc]
        text_filtered = [w.lower() for w in text_tokens if not w.lower() in stop]
        text_stemmed = [stemmer.stem(w) for w in text_filtered]
        # print(title_stemmed)
        # print(text_stemmed)
        titles.append(title_stemmed)
        texts.append(text_stemmed)
        if i % 5000 == 0:
            print(i)
    d = {"title":titles, "text":texts, "label":data["label"]}
    data_cleaned = pd.DataFrame(data=d)
    # data_cleaned["title"] = titles
    # data_cleaned["text"] = texts
    data_cleaned.to_csv("data/data_token.csv")
# tokenize()

## Creating and Training Word2Vec Model

In [275]:
load_model_from_disc = True
w2v_model = None
data = pd.read_csv('data_tokenized/data_token.csv', index_col=0)
# for i, row in data.iterrows():
#     print(type(row["title"]))
#     print(row)
#     data.loc[i, "title"] = ast.literal_eval(row["title"])
#     data.loc[i, "text"] = ast.literal_eval(row["text"])
if load_model_from_disc:
    try:
        w2v_model = Word2Vec.load("word2vec.model")
    except:
        pass

if w2v_model is None or not load_model_from_disc:
    if load_model_from_disc:
        print("Could not load model from disc. Training model...")
    else:
        print("Loading from disc deactivated. Training model...")

    class MySentences(object):
        def __init__(self, data):
            self.data = data

        def __iter__(self):
            for doc in pd.concat([data["text"], data["title"]]): #change to "title" or combine both
                doc = ast.literal_eval(doc)
                yield doc

    sentences = MySentences(data)

    cores = multiprocessing.cpu_count()
    w2v_model = Word2Vec(min_count=20,
                         window=2,
                         sample=6e-5,
                         alpha=0.03,
                         min_alpha=0.0007,
                         negative=20,
                         workers=cores-1)

    w2v_model.build_vocab(sentences, progress_per=10000)
    t = time.time()
    w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=3, report_delay=1)
    print('Time to train the model: {} mins'.format(round((time.time() - t) / 60, 2)))
    w2v_model.save("word2vec.model")
else:
    print("Model loaded from disc.")

Model loaded from disc.


In [276]:
# calculate similarity
w2v_model.wv.similarity("amazon", 'nazi')

-0.11904364

In [277]:
# calculate similarity
w2v_model.wv.similarity("obama", 'trump')

0.60530704

In [278]:
# find out which element doesn't match
w2v_model.wv.doesnt_match(['amazon', 'obama', 'trump'])

'amazon'

In [279]:
# Which word is to obama as georg is to bush?
w2v_model.wv.most_similar(positive=["obama", "georg"], negative=["bush"], topn=3)

[('barack', 0.607993483543396),
 ('presid', 0.4728606641292572),
 ('behest', 0.4636874198913574)]

In [280]:
# e.g. words most similar to obama
w2v_model.wv.most_similar(positive=["obama"])

[('barack', 0.832769513130188),
 ('administr', 0.6584988832473755),
 ('presid', 0.6397863626480103),
 ('predecessor', 0.625878632068634),
 ('trump', 0.6053071022033691),
 ('bush', 0.557529628276825),
 ('outgo', 0.5535241961479187),
 ('undo', 0.5471165180206299),
 ('holdov', 0.5300476551055908),
 ('clinton', 0.5224902629852295)]

In [281]:
# e.g. words most similar to obama
w2v_model.wv.most_similar(positive=["presid"])

[('barack', 0.7051833868026733),
 ('45th', 0.6659713387489319),
 ('successor', 0.6399291157722473),
 ('obama', 0.6397863626480103),
 ('administr', 0.6288126111030579),
 ('trump', 0.6117547750473022),
 ('donald', 0.6109979748725891),
 ('predecessor', 0.6096833348274231),
 ('pres', 0.594482958316803),
 ('presidenti', 0.5868942737579346)]

## Creating Doc2Vec
word2vec for each word with average over document

In [282]:
# creates w2v representation for all documents and titles
def doc2vec():
    titles = list()
    texts = list()
    start = time.time()
    for i, row in data.iterrows():
        vec_title = np.zeros(shape=w2v_model.vector_size)
        vec_text = np.zeros(shape=w2v_model.vector_size)
        tit = ast.literal_eval(row["title"])
        tex = ast.literal_eval(row["text"])
        tit_cnt = 0
        tex_cnt = 0
        for word in tit:
            try:
                vec_title += w2v_model.wv[word]
            except KeyError:
                # print("Didn't find word {}".format(word))
                tit_cnt += 1
                pass
        for word in tex:
            try:
                vec_text += w2v_model.wv[word]
            except KeyError:
                # print("Didn't find word {}".format(word))
                tex_cnt += 1
                pass
        if len(tit) > tit_cnt:
            vec_title /= (len(tit) - tit_cnt)
        if len(tex) > tex_cnt:
            vec_text /= (len(tex) - tex_cnt)
        titles.append(vec_title.tolist())
        texts.append(vec_text.tolist())
        if i % 5000 == 0:
            print("[{}/{}] - {:.1f}s".format(i, len(data.index), time.time() - start))
    end = time.time()
    print("creating doc2vec took {:.1f}s".format(end - start))
    d = {"title":titles, "text":texts, "label":data["label"]}
    data_w2v = pd.DataFrame(data=d)
    data_w2v.to_csv("data/data_w2v.csv")
    display(data_w2v[:100])
doc2vec()

[0/70735] - 0.0s
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[ 0.99748569  0.2247427   0.4354149   0.19536692 -0.2018244   0.09970292
  0.55635818  0.61088441  0.65475376 -0.27401805 -0.61830201 -1.27911472
 -0.45447664 -0.01037367 -0.39624905 -0.71380613 -0.2468091  -1.00076924
  0.50735812  0.12101652  0.85197233 -0.63821731 -0.23425124  0.15814104
  0.64235136 -0.17906579  0.43393953  0.62042567 -0.9378583   0.10757749
 -0.49557299  0.36399874 -0.55203386 -0.11273773  0.57342574 -0.16634555
  0.70404322  0.92241394  0.55667547 -0.20816373 -0.25437867 -0.02401135
 -1.15483778 -0.62305382  0.13446344  0.76675004 -0.39184166  1.03597337
 -0.56808898 -0.11320374  0.56671156 -0.22944351 -0.58812296 -1.10388675
 -0.75314739  0.3267

,title,text,label
0,"[-0.17737161740660667, 0.23486142853895822, 0.12445201848944028, -0.36334328601757687, 0.3301913384348154, -0.028877111462255318, 0.3542606706420581, -0.1827598474919796, -0.36685079149901867, 0.17834910843521357, 0.2996298174063365, -0.34684255595008534, -0.13291914388537407, 0.5066589800020059, 0.19267991185188293, -0.2708385114868482, -0.017944110557436943, 0.28307173525293666, -0.11388187358776729, -0.30716870352625847, 0.41513919038698077, 0.7277844523390135, -0.06439176139732201, 0.09227927153309186, -0.5425457172095776, 0.14019572952141365, 0.029552965114514034, -0.4386397817482551, -0.42108624925216037, -0.413217222938935, -0.13197220209985971, 0.21986744770159325, -0.2825335699599236, -0.16889805952087045, -0.4155910698076089, 0.2879291175243755, -0.1103774116684993, -0.8907897769434688, -0.3223227026561896, 0.10792573584088434, -0.1445204878691584, -0.2927070196407537, 0.2897471431642771, 0.10879019275307655, 0.47844405348102254, -0.44211805953333777, 0.5719812869404753, -0.4012464658978085, 0.7594906856926779, -0.1151490987977013, 0.14371274524213126, 0.22255045330772796, -0.1222666905572017, 0.4955827960123618, -0.31929607689380646, 0.017945933466156323, 0.16614588738108674, -0.01917497751613458, -0.07837012064798425, -0.35003646835684776, 0.03681470391650995, 0.2148221828926277, 0.5458240484197935, -0.37602727115154266, -0.15628574431563416, 0.0708180017148455, -0.18581471219658852, 0.08098900131881237, -0.539757615265747, 0.5942123178392649, 0.3621960940460364, 0.18527882150374353, 0.20178120645383993, -0.012844864279031754, -0.5178411702315012, 0.5746897514909506, -0.21704354463145137, -0.5987081391115984, -0.42566451740761596, -0.23372547763089338, 0.31736567585418624, -0.20182241996129355, -0.7591941518088182, 0.20308093664546809, 0.03316183264056841, 0.42153524021462846, -0.08880513596038024, -0.13754549156874418, 0.5791262040535609, -0.3480027320717151, 0.28539641325672466, 0.2626928749183814, 0.11944863498987009, 0.2772822293142478, 0.15702432145675024, 0.49627114459872246, 0.4761998246734341, -0.2544484054669738, -0.09271977220972379, -0.5678285965695977]","[-0.40317140313859323, 0.17912740358198212, 0.050562859989638054, 0.055954877193714135, 0.0934258276801718, -0.023519680137824826, 0.24632954259287898, 0.1434498220579113, -0.23731825938003862, 0.017518557575046452, 0.02849149547639523, -0.33533329706499654, -0.2550358505249872, 0.37727573531560615, -0.23844510968698798, -0.0501059377303351, 0.00435799360805577, -0.042552340270672596, -0.07153317957728648, -0.28219471891853076, 0.316290018046194, 0.3761055734737851, 0.07512606074023434, 0.12298925005307587, -0.2549261285887057, 0.14509551490984307, -0.13024391400400012, -0.3689671294372771, -0.086444049350384, -0.15495215342814456, 0.29306362660242136, 0.28684928263699205, -0.2653720033543948, -0.1925096292161825, -0.4215396416248297, -0.043265524127373146, -0.1471679153010742, -0.6697407869658475, -0.12966115223728686, -0.30731571397677576, -0.08219364198473224, -0.20401955047391765, 0.10506988533594748, 0.12025182841066877, 0.12918642091053717, -0.21510587996700192, 0.37280161471516216, -0.4425061703104033, 0.2538979571554335, 0.11037910568357169, -0.007000621740045653, -0.08913956716329156, 0.08515967715328555, 0.27757835686267424, -0.22118830430898295, -0.07811686954735789, 0.05627476818376744, 0.08087808955834527, -0.0853087707212318, -0.1422012611342808, 0.17385939709284623, 0.2476592813871009, -0.041187109410764804, 0.1138095081755548, -0.05519140118691634, -0.014071714172418554, -0.0672952887194823, 0.12840113050306626, -0.3247153297708551, 0.10708220859983464, 0.07283706835182202, 0.0888161897662332, 0.24702526510182016, 0.09409497735250151, -0.11882258266556014, 0.2885739945335851, 0.11462983908889302, -0.11191882241876905, -0.21861281287083664, -0.01776596985215931, -0.06559378575228575, 0.03072875509858778, -0.548379166946704, 0.3558431576434654, -0.08911928602538745, 0.007791372722645094, -0.0045858592184113015, -0.2569

## Train-Test-split and Dataloader Creation

In [283]:
import ast
import string

import torch
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from gensim.models import Word2Vec
import multiprocessing
import time
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

params = {'batch_size': 10,
          'shuffle': True,
          'num_workers': 0}
max_epochs = 100


data_d2v = pd.read_csv("data/data_w2v.csv", index_col=0)
titles = list()
texts = list()
for i, row in data_d2v.iterrows():
    titles.append(ast.literal_eval(row["title"]))
    texts.append(ast.literal_eval(row["text"]))

data_d2v["title"] = titles
data_d2v["text"] = texts


data_d2v_title = data_d2v[["title", "label"]].copy()
data_d2v_text = data_d2v[["text", "label"]].copy()
X_train_title, X_test_title, y_train_title, y_test_title = train_test_split(data_d2v_title["title"], data_d2v_title["label"], test_size=0.15, random_state=42, shuffle=True)
X_train_text, X_test_text, y_train_text, y_test_text = train_test_split(data_d2v_text["text"], data_d2v_text["label"], test_size=0.15, random_state=42, shuffle=True)

X_train_title.reset_index(drop=True, inplace=True)
X_test_title.reset_index(drop=True, inplace=True)
y_train_title.reset_index(drop=True, inplace=True)
y_test_title.reset_index(drop=True, inplace=True)
X_train_text.reset_index(drop=True, inplace=True)
X_test_text.reset_index(drop=True, inplace=True)
y_train_text.reset_index(drop=True, inplace=True)
y_test_text.reset_index(drop=True, inplace=True)


class data_set(Dataset):
    def __init__(self, X, y):
        super(Dataset, self).__init__()
        assert len(X.index) == len(y.index)
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X.index)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

train_dataset_title = data_set(X_train_title, y_train_title)
test_dataset_title = data_set(X_test_title, y_test_title)
train_dataset_text = data_set(X_train_text, y_train_text)
test_dataset_text = data_set(X_test_text, y_test_text)

train_dataloader_title = DataLoader(train_dataset_title, **params)
test_dataloader_title = DataLoader(test_dataset_title, **params)
train_dataloader_text = DataLoader(train_dataset_text, **params)
test_dataloader_text = DataLoader(test_dataset_text, **params)


print("bf")
for batch, (X, y) in enumerate(train_dataloader_title):

    print(X)
    print(y)
    break

print("done")


1923
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.9974856872594994, 0.22474270370852856, 0.4354148988562231, 0.19536691727756544, -0.20182440283473665, 0.09970291618627446, 0.5563581765465836, 0.6108844055019262, 0.6547537642886675, -0.2740180529654026, -0.6183020116420079, -1.2791147189968415, -0.4544766436522926, -0.010373667276416415, -0.39624904681935563, -0.7138061252714113, -0.24680909886956215, -1.0007692363880973, 0.5073581156925163, 0.12101651515495317, 0.8519723341437696, -0.6382173126890459, -0.23425124359551733, 0.158141043